<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Nicolás Gatica
- Nombre de alumno 2: Cristóbal Igor


### **Link de repositorio de GitHub:** `https://github.com/ngatica/LaboratoriosMDS`

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Utilizar la API de Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Utilizar `cache_lru` para cachear resultados de funciones.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Medir el tiempo de ejecución como también el uso de memoria de la función anterior.
- Optimizar la función anterior a través de `numba`.

#Importamos librerias utiles 😸

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%capture
!pip install praw
!pip install line_profiler
!pip install numba

%load_ext line_profiler
%load_ext autoreload
%autoreload 2

import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://preview.redd.it/3sxusxcjge791.png?auto=webp&s=88bc1f9a3c59eafda24b0e32dd87d7cf596d205d">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [3]:
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url


In [4]:
votes, post, url = praw_reddit()

### Estudiando los tiempos:

In [5]:
%lprun -f praw_reddit praw_reddit()

Timer unit: 1e-07 s

Total time: 18.4988 s
File: <ipython-input-3-bd47716803a4>
Function: praw_reddit at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     2         2       5006.0   2503.0      0.0      reddit = praw.Reddit(
     3         1          8.0      8.0      0.0          client_id="-w2hyFINxZ8T3g",
     4         1          4.0      4.0      0.0          client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
     5         1          4.0      4.0      0.0          password="ClasesMDS7202",
     6         1          4.0      4.0      0.0          user_agent="Clases",
     7         1          4.0      4.0      0.0          username="DocenciaDataScience",
     8         1          5.0      5.0      0.0          check_for_async=False,
     9                                               )
    10         1        178.0    178.0      0.0      subreddit = reddi

### Estudiando memoria

In [6]:
!pip install -q memory_profiler
%load_ext memory_profiler

In [7]:
%memit praw_reddit()

peak memory: 197.81 MiB, increment: 5.57 MiB


Creando modulo:

In [8]:
%%file praw_reddit_module.py
import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

Overwriting praw_reddit_module.py


In [9]:
from praw_reddit_module import praw_reddit
%mprun -f praw_reddit praw_reddit()

Filename: c:\Users\ngati\Downloads\praw_reddit_module.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     4    194.1 MiB    194.1 MiB           1   def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     5    194.1 MiB      0.0 MiB           2       reddit = praw.Reddit(
     6    194.1 MiB      0.0 MiB           1           client_id="-w2hyFINxZ8T3g",
     7    194.1 MiB      0.0 MiB           1           client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
     8    194.1 MiB      0.0 MiB           1           password="ClasesMDS7202",
     9    194.1 MiB      0.0 MiB           1           user_agent="Clases",
    10    194.1 MiB      0.0 MiB           1           username="DocenciaDataScience",
    11    194.1 MiB      0.0 MiB           1           check_for_async=False,
    12                                             )
    13    194.1 MiB      0.0 MiB           1       subreddit = reddit.subreddit(nombre_subreddit)
    14                                        

**Respuesta:**

```Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.```

En cuanto a los tiempos de la función, el primer proceso que más tarda en la ejecución del código corresponde a la obtención y creación de la lista de los n_hot posts en el subreddit de Chile, lo cual puede ser mejorado si es que existe alguna forma de convertir a lista las submisiones de forma más eficiente (dependerá de la librería) o guardar esta misma en la memoria cache.

Luego, lo segundo que más tarda, es al momento de trabajar con las submisiones en un doble ciclo de for's, sobre todo al ejecutar el segundo for que calcula la cantidad de upvotes de cada submisión (la forma de calcular upvotes es lo que más tarda dada la utilización de if else). Es este doble for lo que se puede mejorar tratando de aplicar las operaciones de forma paralela para todos los valores al aprovechar las transformaciones que se pueden realizar a un array.

```Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?```

En cuanto a la memoria, se puede observar un uso de memoria en torno a 230 MB, en dónde la mayor parte esta asociada a la ejecución completa de la función, teniendo sólamente un incremento de 9MB en la creación de la lista de los n_hot posts en el subreddit de Chile, que tambien corresponde a lo que toma más tiempo dentro de la función, por lo que el principal objetivo sería tratar esta línea de ejecución.

## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

**Respuestas Teóricas:**

`Escriba su respuesta aquí`
1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?

R: La memoria cache es una memoria que permite almacenar en el disco o en la RAM aquellos resultados de procesos previos con la finalidad de ser utilizados más adelante. Las siglas LRU hacen referencia a la oración en inglés Least Recently Used, y esto se refiere principalmente al orden de ejecución en el cual se almacenarán los procesos en la memoria cache.

2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?

R: Los costos que tiene la aplicación de técnicas de Caching vienen principalmente relacionadas al aumento en el uso de la memoria. Esto puede conllevar problemas debido a que la memoria cuenta con ciertas limitaciones dadas por las especificaciones de cada equipo.

3.	¿Cuál es la consecuencía de ocupar caching en la función anterior?.

R: Dentro de la función anterior el uso de memoria se encontraba cerca de los 230 MB y el uso de técnicas de caching aumentaría el uso en memoria cache con el fin de acceder de forma más rápida a la lista. Esto trae como consecuencia que se ocupa una cantidad no mínima de su espacio la cual no queda disponible para ser utilizado en otras ejecuciones.

In [19]:
###### Código Aquí ######
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
        
    subreddit  = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit = n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions),50)):
        top_n_submissions = top_submissions[:top_n]
        
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [11]:
%%timeit
praw_reddit()

18.1 s ± 267 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
@lru_cache
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
        
    subreddit  = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit = n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions),50)):
        top_n_submissions = top_submissions[:top_n]
        
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [15]:
%lprun -f praw_reddit praw_reddit()

Timer unit: 1e-07 s

In [37]:
%%timeit
praw_reddit()

The slowest run took 6.50 times longer than the fastest. This could mean that an intermediate result is being cached.
443 ns ± 354 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Respuesta de lru_cache sobre praw_reddit**

`Como se ve, al utilizar LRU cache se baja el tiempo de ejecución de segundos a menos de un segundo,esto es debido a que este decorador permite memorizar las submisiones con las que se trabajan y que era lo que más tardaba en un inicio.`

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [31]:
###### Código Aquí ######
import math 
def intervalos(u, d):
    mu = []
    sigma = []
    for i in range(len(u)):
        a = 1 + u[i]
        b = 1 + d[i]
        sigma.append(1.65 * math.sqrt(a*b/(((a+b)**2)*(a+b+1))))
        mu.append(a/(a+b))
    return mu,sigma
    


def intervalos_numpy(u, d):
    a = 1 + u
    b = 1 + d
    sigma = 1.65 * np.sqrt(a*b/(((a+b)**2)*(a+b+1)))
    mu = a/(a+b)
    return mu,sigma

In [33]:
votes_in = votes[1]

In [8]:
%timeit intervalos(votes_in[:, 0], votes_in[:, 1])

151 µs ± 784 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [23]:
%timeit intervalos_numpy(votes_in[:, 0], votes_in[:, 1])

11.1 µs ± 110 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [24]:
intervalos(votes_in[:, 0], votes_in[:, 1])

([0.5398230088495575,
  0.8175675675675675,
  0.9463414634146341,
  0.9021739130434783,
  0.9732142857142857,
  0.9117647058823529,
  0.9111111111111111,
  0.9473684210526315,
  0.9166666666666666,
  0.9384615384615385,
  0.927710843373494,
  0.8640776699029126,
  0.8258426966292135,
  0.8461538461538461,
  0.8235294117647058,
  0.8860103626943006,
  0.9508196721311475,
  0.9483568075117371,
  0.875,
  0.875,
  0.8974358974358975,
  0.9786200194363459,
  0.9763513513513513,
  0.8222222222222222,
  0.7692307692307693,
  0.5982142857142857,
  0.8611111111111112,
  0.9454545454545454,
  0.8571428571428571,
  0.8333333333333334,
  0.75,
  0.7142857142857143,
  0.9557522123893806,
  0.75,
  0.9887323943661972,
  0.8878048780487805,
  0.9183098591549296,
  0.6666666666666666,
  0.9047619047619048,
  0.9278350515463918,
  0.8157894736842105,
  0.9803921568627451,
  0.9379699248120301,
  0.5454545454545454,
  0.7161290322580646,
  0.5625,
  0.8,
  0.7142857142857143,
  0.8867924528301887,
  0.

In [32]:
intervalos_numpy(votes_in[:, 0], votes_in[:, 1])

(array([0.53982301, 0.81756757, 0.94634146, 0.90217391, 0.97321429,
        0.91176471, 0.91111111, 0.94736842, 0.91666667, 0.93846154,
        0.92771084, 0.86407767, 0.8258427 , 0.84615385, 0.82352941,
        0.88601036, 0.95081967, 0.94835681, 0.875     , 0.875     ,
        0.8974359 , 0.97862002, 0.97635135, 0.82222222, 0.76923077,
        0.59821429, 0.86111111, 0.94545455, 0.85714286, 0.83333333,
        0.75      , 0.71428571, 0.95575221, 0.75      , 0.98873239,
        0.88780488, 0.91830986, 0.66666667, 0.9047619 , 0.92783505,
        0.81578947, 0.98039216, 0.93796992, 0.54545455, 0.71612903,
        0.5625    , 0.8       , 0.71428571, 0.88679245, 0.57746479,
        0.77647059, 0.58333333, 0.95424837, 0.91836735, 0.625     ,
        0.57142857, 0.65714286, 0.92753623, 0.87755102, 0.90697674,
        0.81578947, 0.7826087 , 0.9375    , 0.53333333, 0.875     ,
        0.8951049 , 0.81818182, 0.94174757, 0.89189189, 0.52631579,
        0.80451128, 0.8372093 , 0.78431373, 0.72

## 1.4 Comparación de Rendimiento [2 Puntos]

Ahora, aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

Para esto genere un gráfico de linea utilizando plotly, en donde se pueda observar el tiempo que toma ejecutar cada función (el eje y) diferentes cantidades de datos (ejemplo, en el eje x considerar 100, 200, 300, ..., 1000 datos) sobre cada función implementada (series `Python`, `Numpy`, `Python-JIT`, `Numpy-JIT`). ¿Es posible observar diferencias? ¿A qué se debe esto?.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

**Nota 2**: En el caso que su computador sea muy rápido, se recomienda graficar el eje y (tiempo) como logaritmo usando `log_y=True`.


In [29]:
###### Código Aquí ######
import math 
@jit(nopython=True)
def intervalos_JIT_py(u, d):
    mu = []
    sigma = []
    for i in range(len(u)):
        a = 1 + u[i]
        b = 1 + d[i]
        sigma.append(1.65 * math.sqrt(a*b/(((a+b)**2)*(a+b+1))))
        mu.append(a/(a+b))
    return mu,sigma
    

    
@jit(nopython=True)
def intervalos_numpy_JIT(u, d):
    a = 1 + u
    b = 1 + d
    sigma = 1.65 * np.sqrt(a*b/(((a+b)**2)*(a+b+1)))
    mu = a/(a+b)
    return mu,sigma

In [ ]:
intervalos_JIT_py(votes_in[:, 0], votes_in[:, 1])

In [ ]:
intervalos_numpy_JIT(votes_in[:, 0], votes_in[:, 1])

In [35]:
from timeit import timeit
tiempos_py = []
tiempos_numpy = []
tiempos_JIT_py = []
tiempos_numpy_JIT = []
for i in range(0, 18, 1):
    t_i = timeit(lambda: intervalos(votes[i][:, 0], votes[i][:, 1]), number=1)
    tiempos_py.append(t_i)
    t_i = timeit(lambda: intervalos_numpy(votes[i][:, 0], votes[i][:, 1]), number=1)
    tiempos_numpy.append(t_i)
    t_i = timeit(lambda: intervalos_JIT_py(votes[i][:, 0], votes[i][:, 1]), number=1)
    tiempos_JIT_py.append(t_i)
    t_i = timeit(lambda: intervalos_numpy_JIT(votes[i][:, 0], votes[i][:, 1]), number=1)
    tiempos_numpy_JIT.append(t_i)


In [36]:
print(tiempos_py)
print(tiempos_numpy)
print(tiempos_JIT_py)
print(tiempos_numpy_JIT)

[0.00010009999994053942, 0.0001611000000139029, 0.00022850000004837057, 0.0003048999999464286, 0.00038530000006176124, 0.00045709999994869577, 0.0005316000000448184, 0.0006118000000014945, 0.0008589999999912834, 0.0008730999999215783, 0.0008289999999533393, 0.0009369999999080392, 0.0009767999999894528, 0.001064400000018395, 0.0011193999999932203, 0.0013639999999668362, 0.0014309000000594096, 0.0013899000000492379]
[3.9100000094549614e-05, 3.270000001975859e-05, 3.049999997983832e-05, 1.940000004196918e-05, 2.4600000074315176e-05, 2.329999995254184e-05, 2.1800000013172394e-05, 2.6000000048043148e-05, 4.139999998642452e-05, 3.570000001218432e-05, 3.090000006977789e-05, 3.0700000024808105e-05, 3.249999997478881e-05, 2.649999998993735e-05, 3.049999997983832e-05, 5.8399999943503644e-05, 4.6600000018770515e-05, 4.7399999971275975e-05]
[0.15931330000000798, 1.3699999954042141e-05, 1.499999996212864e-05, 2.090000009502546e-05, 2.0999999946980097e-05, 2.7700000032382377e-05, 2.4600000074315176e

In [37]:
x = list(range(50, 901, 50))

In [38]:
import pandas as pd
data = {'Datos' : x,
        'Tiempos_py': tiempos_py,
        'Tiempos_numpy': tiempos_numpy,
        'Tiempos_JIT_py': tiempos_JIT_py,
        'Tiempos_numpy_JIT': tiempos_numpy_JIT}
df = pd.DataFrame(data)
df

,Datos,Tiempos_py,Tiempos_numpy,Tiempos_JIT_py,Tiempos_numpy_JIT
0,50,0.000100,0.000039,0.159313,0.000013
1,100,0.000161,0.000033,0.000014,0.000006
2,150,0.000229,0.000030,0.000015,0.000005
3,200,0.000305,0.000019,0.000021,0.000006
4,250,0.000385,0.000025,0.000021,0.000011
5,300,0.000457,0.000023,0.000028,0.000010
6,350,0.000532,0.000022,0.000025,0.000009
7,400,0.000612,0.000026,0.000030,0.000009
8,450,0.000859,0.000041,0.000039,0.000010
9,500,0.000873,0.000036,0.000040,0.000016


In [49]:
px.line(df, y=['Tiempos_py', 'Tiempos_numpy', 'Tiempos_JIT_py', 'Tiempos_numpy_JIT'], x='Datos', log_y=True)

Es posible observar diferencias? A qué se debe esto?

`
Dentro de los gráficos es claro notar las diferencias que existen en 3 partes principales, primero los tiempos de ejecución de la función sin numpy la cual naturalmente es la que más tarda (utiliza funciones iterativas como el for), luego está la función con numpy que tarda menos que la primera al ahorrarse los ciclos iterativos del for mediante el uso de la librería, y finalmente se encuentran las funciones compiladas con Numba el cual es un compilador diseñado para optimizar tareas matemáticas y con ciclos, por lo cual es esperado que estas funciones compilen en un timepo mucho menor al resto.
`

## 1.5 Plot de Resultados de Análisis de Votos [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [44]:
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
posterior_mean, std_err = intervalos_numpy(votos[:, 0], votos[:, 1])
lb = posterior_mean - std_err
print(lb)
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post
df

lower bounds aproximados:
[0.48523979 0.78059168 0.92043586 0.85134452 0.94815316 0.83265815
 0.84187791 0.9053808  0.85827725 0.88965326 0.88108924 0.80862926
 0.77907167 0.73158407 0.7172068  0.84836295 0.90550562 0.93068575
 0.76586276 0.74265165 0.81828545 0.97118339 0.96180306 0.72921035
 0.6354421  0.52211679 0.7673017  0.9202495  0.7609129  0.60091591
 0.57671534 0.45074913 0.93323112 0.51184301 0.98220102 0.86211815
 0.89435805 0.3726793  0.85771798 0.88470605 0.71336637 0.95785079
 0.92073077 0.30828372 0.65656587 0.36397748 0.72429281 0.52182581
 0.81564887 0.48141157 0.70234561 0.35771985 0.92646674 0.87296205
 0.35873239 0.28273871 0.5641946  0.87640799 0.8010596  0.87939801
 0.71336637 0.68333632 0.91574322 0.32754218 0.81715738 0.85297239
 0.70904079 0.90385173 0.80877728 0.34209586 0.74798395 0.74537828
 0.69020315 0.58949612 0.91113817 0.395419   0.02368603 0.61531815
 0.02368603 0.58960959 0.85719803 0.79473599 0.95354137 0.59123643
 0.63447009 0.80149882 0.85847271 0.

,Votos (+),Votos (-),Post,url
0,272,0,Juraba que esta wea era un sueño febril de chico,https://i.redd.it/zvyt2znkta891.jpg
1,235,0,Gobierno queda fuera de querella por asesinato...,https://www.eldinamo.cl/pais/Gobierno-queda-fu...
2,226,0,Empresa le paga por error sueldo de $ 165 mill...,https://www.t13.cl/noticia/diario-financiero/n...
3,214,0,Macabro hallazgo en San José de Maipo: perro l...,https://www.biobiochile.cl/noticias/nacional/r...
4,724,7,ahora no po',https://i.redd.it/83odxdkaq9791.png
5,773,8,Bolos,https://i.redd.it/8tx19xd0k2691.jpg
6,701,7,Pudú en mi u 🤠,https://i.redd.it/iux9a5xlvr891.jpg
7,524,5,la situación de los pudúes en Chiloé es crític...,https://www.reddit.com/gallery/vhblmb
8,430,4,Otros tiempos,https://i.redd.it/3dgqvverce791.jpg
9,478,5,Con razón hay tanto taco,https://i.redd.it/nf9c4wh63g791.jpg


In [40]:
r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()

**Respuesta:**

`Qué logra observar? La solución resulta trivial? `

En base al dataframe se puede ver que los posts más populares son aquellos post que tienen 0 votos negativos, lo cual puede estar boosteando de sobremanera las submisiones de reddit por sobre otras que tienen de 3 a 4 veces más votos, pero que por unos pocos votos negativos se encuentran con una popularidad menor. Un ejemplo de esto son los posts que se encuentra en la 4ta, 5ta y 6ta posición, los cuales tienen entre 700 y 800 votos positivos por lo que naturalmente se pensaría que son más populares que los que se encuentran en los primeros lugares, los cuales no superan los 280 votos positivos.

En este sentido se cree que la solución no resulta trivial dado que el algoritmo que se está implementando para la obtención de las medias no está tomando en consideración la proporción o magnitud de votos positivos por sobre los votos negativos de la mejor manera, afectando esto en el std_error otorgando por ejemplo diferencias muy grandes entre los valores de min y max en aquellos post que cuentan con una menor cantidad de votos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>